## **Setup**

Buatlah file Google Spreadsheet di personal account Google Drive. Import data dari URL Google Spreadsheet Aruna.



> Menggunakan fungsi IMPORTRANGE



Link Google Spreadsheet : 

<b> https://docs.google.com/spreadsheets/d/1mYPXtMXQ8-wvwBq0WsDjDTUqIGElSVl1mdDuhSWGsdI/edit?usp=sharing 

Extract and Transformation dari Google Sheet menjadi files dengan parquet extension sesuai dengan transaction type.

## **Data Preparation**

Sheet Name:
- BKJ_Delivery
- BKJ_Buying
- BKJ_Processing
- DOB_Delivery
- DOB_Buying
- DOB_Processing

Import Library

In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
!pip install fastparquet

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.6 MB 3.8 MB/s 
     |████████████████████████████████| 12.2 MB 28.5 MB/s 
     |████████████████████████████████| 2.0 MB 27.1 MB/s 
  Attempting uninstall: pandas
    Found existing installation: pandas 1.3.5
    Uninstalling pandas-1.3.5:
      Successfully uninstalled pandas-1.3.5


In [3]:
import pandas as pd
import pyarrow
import fastparquet

In [4]:
 pd.pandas.set_option('display.max_columns', None)

## **Data Extraction, Data Transformation and Data Cleaning**

Data Extraction from Google Spreadsheet

In [5]:
sheet_id = '1mYPXtMXQ8-wvwBq0WsDjDTUqIGElSVl1mdDuhSWGsdI'
sheet_name = ['BKJ_Delivery', 'BKJ_Buying', 'BKJ_Processing', 'DOB_Delivery', 'DOB_Buying', 'DOB_Processing']

BKJ_Delivery = f"https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet={sheet_name[0]}"
BKJ_Buying = f"https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet={sheet_name[1]}"
BKJ_Processing = f"https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet={sheet_name[2]}"
DOB_Delivery = f"https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet={sheet_name[3]}"
DOB_Buying = f"https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet={sheet_name[4]}"
DOB_Processing = f"https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet={sheet_name[5]}"

### **BKJ Delivery**

In [6]:
#Read data csv
df = pd.read_csv(BKJ_Delivery)

In [7]:
#Melihat nama kolom
df.columns

Index(['Norm Helper', 'Delivery Number', 'Tanggal Delivery dari MP',
       'Buyer Name', 'Product Kirim', 'Tanggal Diterima Buyer', 'Unnamed: 6'],
      dtype='object')

In [8]:
#Data cleaning : menghapus kolom 'Unnamed: 6'
df = df.drop('Unnamed: 6', axis=1)
df.columns

Index(['Norm Helper', 'Delivery Number', 'Tanggal Delivery dari MP',
       'Buyer Name', 'Product Kirim', 'Tanggal Diterima Buyer'],
      dtype='object')

In [9]:
#Data transformation : mengubah extention file yang semula CSV menjadi Parquet
df.to_parquet('BKJ_Delivery.parquet', index=False)
#Menyimpan file parquet di Google Drive
!cp BKJ_Delivery.parquet "drive/My Drive/"

In [10]:
#Read data parquet
df_BKJ_Delivery = pd.read_parquet('BKJ_Delivery.parquet')

In [11]:
#Preview data
df_BKJ_Delivery.head()

,Norm Helper,Delivery Number,Tanggal Delivery dari MP,Buyer Name,Product Kirim,Tanggal Diterima Buyer
0,DO/BKJ/20220305/001/1,1,05/03/2022,PT Prima,Tuna (Whole),06/03/2022
1,DO/BKJ/20220307/002/1,2,07/03/2022,PT Prima,Tuna (Whole),08/03/2022
2,DO/BKJ/20220309/003/1,3,09/03/2022,PT Prima,Tuna (Whole),10/03/2022


In [12]:
#Ukuran data (jumlah baris, jumlah kolom)
df_BKJ_Delivery.shape

(3, 6)

In [13]:
#Summary data info
df_BKJ_Delivery.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3 entries, 0 to 2
Data columns (total 6 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   Norm Helper               3 non-null      object
 1   Delivery Number           3 non-null      int64 
 2   Tanggal Delivery dari MP  3 non-null      object
 3   Buyer Name                3 non-null      object
 4   Product Kirim             3 non-null      object
 5   Tanggal Diterima Buyer    3 non-null      object
dtypes: int64(1), object(5)
memory usage: 272.0+ bytes


Kesimpulan :

---
- Tidak ada missing value (data hilang/ data kosong)
- Kolom `Tanggal Delivery dari MP` dan `Tanggal Diterima Buyer` memiliki tipe data yang salah yaitu object, seharusnya berformat datetime


In [14]:
#Data transformation : mengubah tipe data menjadi datetime
df_BKJ_Delivery['Tanggal Delivery dari MP'] = pd.to_datetime(df_BKJ_Delivery['Tanggal Delivery dari MP'], dayfirst=True)
df_BKJ_Delivery['Tanggal Diterima Buyer'] = pd.to_datetime(df_BKJ_Delivery['Tanggal Diterima Buyer'], dayfirst=True)

In [15]:
#Check tipe data
df_BKJ_Delivery.dtypes

Norm Helper                         object
Delivery Number                      int64
Tanggal Delivery dari MP    datetime64[ns]
Buyer Name                          object
Product Kirim                       object
Tanggal Diterima Buyer      datetime64[ns]
dtype: object

In [16]:
#Check duplicated data
df_BKJ_Delivery.duplicated(subset='Norm Helper', keep='first').sum()

0

### **BKJ Buying**

In [17]:
#Read data csv
df2 = pd.read_csv(BKJ_Buying)

In [18]:
#Melihat nama kolom
df2.columns

Index(['PO Number', 'Date Helper', 'Norm Helper', 'Date', 'Month', 'Year',
       'Nama Nelayan/Supplier', 'Type', 'Source Place', 'Category', 'Items',
       'Size', 'Code', 'Quantity', 'Kupon', 'Price', 'Amount', 'ID Nelayan',
       'Sequence 1', 'Sequence 2'],
      dtype='object')

In [19]:
#Data transformation : mengubah extention file yang semula CSV menjadi Parquet
df2.to_parquet('BKJ_Buying.parquet')
#Menyimpan file parquet di Google Drive
!cp BKJ_Buying.parquet "drive/My Drive/"

In [20]:
#Read data parquet
df_BKJ_Buying = pd.read_parquet('BKJ_Buying.parquet')

In [21]:
#Preview data
df_BKJ_Buying.head()

,PO Number,Date Helper,Norm Helper,Date,Month,Year,Nama Nelayan/Supplier,Type,Source Place,Category,Items,Size,Code,Quantity,Kupon,Price,Amount,ID Nelayan,Sequence 1,Sequence 2
0,PO/BKJ/20220305/001,05/03/2022,DO/BKJ/20220305/001/1,05/03/2022,Mar,2022,Nazar,Mini Plant,Bangka Jaya,Ikan Pelagis,Tuna,NaN,TUNA 20,248,NaN,29.0,10.664.000,NaN,1,1
1,PO/BKJ/20220305/002,05/03/2022,DO/BKJ/20220305/001/1,05/03/2022,Mar,2022,Moza,Supplier,Krueng Mane,Ikan Pelagis,Tuna,NaN,TUNA 20,252,NaN,29.0,10.836.000,NaN,2,2
2,PO/BKJ/20220306/001,06/03/2022,DO/BKJ/20220307/002/1,06/03/2022,Mar,2022,Gam,Supplier,Bangka Jaya,Ikan Pelagis,Tuna,NaN,TUNA 20,30,NaN,29.0,1.290.000,NaN,1,1


In [22]:
#Ukuran data (jumlah baris, jumlah kolom)
df_BKJ_Buying.shape

(3, 20)

In [23]:
#Summary data info
df_BKJ_Buying.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3 entries, 0 to 2
Data columns (total 20 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   PO Number              3 non-null      object 
 1   Date Helper            3 non-null      object 
 2   Norm Helper            3 non-null      object 
 3   Date                   3 non-null      object 
 4   Month                  3 non-null      object 
 5   Year                   3 non-null      int64  
 6   Nama Nelayan/Supplier  3 non-null      object 
 7   Type                   3 non-null      object 
 8   Source Place           3 non-null      object 
 9   Category               3 non-null      object 
 10  Items                  3 non-null      object 
 11  Size                   0 non-null      float64
 12  Code                   3 non-null      object 
 13  Quantity               3 non-null      int64  
 14  Kupon                  0 non-null      float64
 15  Price     

Kesimpulan :

---
- Terdapat missing value (data hilang/ data kosong) pada kolom `Size`, `Kupon`, dan `ID Nelayan`
- Kolom `ID Nelayan` menunjukkan sebuah ID yang dalam database seharusnya memiliki tipe interger
- Kolom `Date Helper` dan `Date` seharusnya berformat datetime
- Kolom `Price` memiliki data yang seharusnya menunjukkan harga dalam ribuan (contoh: 29000)
- Kolom `Amount` yang menunjukkan total harga (quantity*price) memiliki tipe data yang salah yaitu object, seharusnya berformat number (int)


In [24]:
#Data cleaning : mengisi missing value dengan nilai 0 (karena memang tidak diketahui)
df_BKJ_Buying = df_BKJ_Buying.fillna(0)

In [25]:
#Data transformation : mengubah tipe data dari float --> int
df_BKJ_Buying['ID Nelayan'] = df_BKJ_Buying['ID Nelayan'].astype('int64')
df_BKJ_Buying['Price'] = df_BKJ_Buying['Price'].astype('int64')

In [26]:
#Data transformation : mengubah tipe data menjadi datetime
df_BKJ_Buying['Date Helper'] = pd.to_datetime(df_BKJ_Buying['Date Helper'], dayfirst=True)
df_BKJ_Buying['Date'] = pd.to_datetime(df_BKJ_Buying['Date'], dayfirst=True)

In [27]:
#Data transformation : mengubah nilai price dalam bentuk ribuan
numbers = df_BKJ_Buying['Price'].tolist()
df_BKJ_Buying['Price'] = [int(str(num).ljust(5, '0')) for num in numbers]

In [28]:
#Data transformation : mengubah tipe data dari object --> int
df_BKJ_Buying['Amount'] = df_BKJ_Buying['Amount'].str.replace('.','')
df_BKJ_Buying['Amount'] = df_BKJ_Buying['Amount'].astype('int64')

<ipython-input-28-a4c6deace3e9>:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df_BKJ_Buying['Amount'] = df_BKJ_Buying['Amount'].str.replace('.','')


In [29]:
#Untuk menyamakan dengan tipe data pada DOB_Buying
df_BKJ_Buying['Quantity'] = df_BKJ_Buying['Quantity'].astype('float64')

In [30]:
#Preview Data After
df_BKJ_Buying.head()

,PO Number,Date Helper,Norm Helper,Date,Month,Year,Nama Nelayan/Supplier,Type,Source Place,Category,Items,Size,Code,Quantity,Kupon,Price,Amount,ID Nelayan,Sequence 1,Sequence 2
0,PO/BKJ/20220305/001,2022-03-05,DO/BKJ/20220305/001/1,2022-03-05,Mar,2022,Nazar,Mini Plant,Bangka Jaya,Ikan Pelagis,Tuna,0.0,TUNA 20,248.0,0.0,29000,10664000,0,1,1
1,PO/BKJ/20220305/002,2022-03-05,DO/BKJ/20220305/001/1,2022-03-05,Mar,2022,Moza,Supplier,Krueng Mane,Ikan Pelagis,Tuna,0.0,TUNA 20,252.0,0.0,29000,10836000,0,2,2
2,PO/BKJ/20220306/001,2022-03-06,DO/BKJ/20220307/002/1,2022-03-06,Mar,2022,Gam,Supplier,Bangka Jaya,Ikan Pelagis,Tuna,0.0,TUNA 20,30.0,0.0,29000,1290000,0,1,1


In [31]:
#Check tipe data
df_BKJ_Buying.dtypes

PO Number                        object
Date Helper              datetime64[ns]
Norm Helper                      object
Date                     datetime64[ns]
Month                            object
Year                              int64
Nama Nelayan/Supplier            object
Type                             object
Source Place                     object
Category                         object
Items                            object
Size                            float64
Code                             object
Quantity                        float64
Kupon                           float64
Price                             int64
Amount                            int64
ID Nelayan                        int64
Sequence 1                        int64
Sequence 2                        int64
dtype: object

In [32]:
#Check duplicated data
df_BKJ_Buying.duplicated(subset='PO Number', keep='first').sum()

0

### **BKJ Processing**

In [33]:
#Read data csv
df3 = pd.read_csv(BKJ_Processing)

In [34]:
#Melihat nama kolom
df3.columns

Index(['PR No', 'Date helper', 'Norm Helper', 'Date', 'RM', 'RC', '%',
       'Total RC', 'RC Dback', '%.1', 'Total RC Dback', 'Meat', '%.2',
       'Total Meat', 'Picker Price', 'Amount', 'Sequence 1', 'Sequence 2'],
      dtype='object')

In [35]:
#Data transformation : mengubah extention file yang semula CSV menjadi Parquet
df3.to_parquet('BKJ_Processing.parquet')
#Menyimpan file parquet di Google Drive
!cp BKJ_Processing.parquet "drive/My Drive/"

In [36]:
#Read data parquet
df_BKJ_Processing = pd.read_parquet('BKJ_Processing.parquet')

In [37]:
#Preview data
df_BKJ_Processing.head()

,PR No,Date helper,Norm Helper,Date,RM,RC,%,Total RC,RC Dback,%.1,Total RC Dback,Meat,%.2,Total Meat,Picker Price,Amount,Sequence 1,Sequence 2
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [38]:
#Ukuran data (jumlah baris, jumlah kolom)
df_BKJ_Processing.shape

(1, 18)

In [39]:
#Summary data info
df_BKJ_Processing.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 18 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   PR No           0 non-null      float64
 1   Date helper     0 non-null      float64
 2   Norm Helper     0 non-null      float64
 3   Date            0 non-null      float64
 4   RM              0 non-null      float64
 5   RC              0 non-null      float64
 6   %               0 non-null      float64
 7   Total RC        0 non-null      float64
 8   RC Dback        0 non-null      float64
 9   %.1             0 non-null      float64
 10  Total RC Dback  0 non-null      float64
 11  Meat            0 non-null      float64
 12  %.2             0 non-null      float64
 13  Total Meat      0 non-null      float64
 14  Picker Price    0 non-null      float64
 15  Amount          0 non-null      float64
 16  Sequence 1      0 non-null      float64
 17  Sequence 2      0 non-null      float64

Kesimpulan :

---
- Terdapat missing value (data hilang/ data kosong) pada semua kolom
- Akan mengubah tipe data yang masih salah


In [40]:
#Mengisi missing value dengan 0
df_BKJ_Processing = df_BKJ_Processing.fillna(0)

In [41]:
#Data transformation : mengubah tipe data float --> object
df_BKJ_Processing['PR No'] = df_BKJ_Processing['PR No'].astype(str)
df_BKJ_Processing['Norm Helper'] = df_BKJ_Processing['Norm Helper'].astype(str)

In [42]:
#Data transformation : mengubah tipe data float --> int
df_BKJ_Processing['Amount'] = df_BKJ_Processing['Amount'].astype('int64')
df_BKJ_Processing['Sequence 1'] = df_BKJ_Processing['Sequence 1'].astype('int64')
df_BKJ_Processing['Sequence 2'] = df_BKJ_Processing['Sequence 2'].astype('int64')

In [43]:
#Data transformation : mengubah tipe data float --> datetime
df_BKJ_Processing['Date helper'] = pd.to_datetime(df_BKJ_Processing['Date helper'])
df_BKJ_Processing['Date'] = pd.to_datetime(df_BKJ_Processing['Date']) 

In [44]:
#Delete row
df_BKJ_Processing = df_BKJ_Processing.drop(index = [0])

In [45]:
#Preview Data After
df_BKJ_Processing.head()

,PR No,Date helper,Norm Helper,Date,RM,RC,%,Total RC,RC Dback,%.1,Total RC Dback,Meat,%.2,Total Meat,Picker Price,Amount,Sequence 1,Sequence 2


In [46]:
#Check tipe data
df_BKJ_Processing.dtypes

PR No                     object
Date helper       datetime64[ns]
Norm Helper               object
Date              datetime64[ns]
RM                       float64
RC                       float64
%                        float64
Total RC                 float64
RC Dback                 float64
%.1                      float64
Total RC Dback           float64
Meat                     float64
%.2                      float64
Total Meat               float64
Picker Price             float64
Amount                     int64
Sequence 1                 int64
Sequence 2                 int64
dtype: object

In [47]:
#Check duplicated data
df_BKJ_Processing.duplicated(subset='PR No', keep='first').sum()

0

### **DOB Delivery**

In [48]:
#Read data csv
df4 = pd.read_csv(DOB_Delivery)

In [49]:
#Melihat nama kolom
df4.columns

Index(['Norm Helper', 'Delivery Number', 'Tanggal Delivery dari MP',
       'Buyer Name', 'Product Kirim', 'Tanggal Diterima Buyer', 'Unnamed: 6',
       'Unnamed: 7', 'Unnamed: 8', 'Unnamed: 9', 'Unnamed: 10', 'Unnamed: 11',
       'Unnamed: 12', 'Unnamed: 13', 'Unnamed: 14', 'Unnamed: 15',
       'Unnamed: 16', 'Unnamed: 17', 'Unnamed: 18', 'Unnamed: 19',
       'Unnamed: 20', 'Unnamed: 21', 'Unnamed: 22', 'Unnamed: 23',
       'Unnamed: 24', 'Unnamed: 25'],
      dtype='object')

In [50]:
cols_to_drop = [ 'Unnamed: 6', 'Unnamed: 7', 'Unnamed: 8', 'Unnamed: 9', 'Unnamed: 10', 'Unnamed: 11', 'Unnamed: 12', 'Unnamed: 13', 'Unnamed: 14', 'Unnamed: 15', 'Unnamed: 16', 'Unnamed: 17', 'Unnamed: 18', 'Unnamed: 19', 'Unnamed: 20', 'Unnamed: 21', 'Unnamed: 22', 'Unnamed: 23', 'Unnamed: 24', 'Unnamed: 25' ]

In [51]:
#Data cleaning : menghapus semua kolom Unnamed
df4 = df4.drop(cols_to_drop, axis=1)
df4.columns

Index(['Norm Helper', 'Delivery Number', 'Tanggal Delivery dari MP',
       'Buyer Name', 'Product Kirim', 'Tanggal Diterima Buyer'],
      dtype='object')

In [52]:
#Data transformation : mengubah extention file yang semula CSV menjadi Parquet
df4.to_parquet('DOB_Delivery.parquet')
#Menyimpan file parquet di Google Drive
!cp DOB_Delivery.parquet "drive/My Drive/"

In [53]:
#Read data parquet
df_DOB_Delivery = pd.read_parquet('DOB_Delivery.parquet')

In [54]:
#Preview data
df_DOB_Delivery.head(10)

,Norm Helper,Delivery Number,Tanggal Delivery dari MP,Buyer Name,Product Kirim,Tanggal Diterima Buyer
0,DO/DOB/20220104/090/1,90,04/01/2022,SJA Lampung,Rajungan (Meat),05/01/2022
1,DO/DOB/20220105/091/1,91,05/01/2022,SJA Lampung,Rajungan (Meat),06/01/2022
2,DO/DOB/20220106/092/1,92,06/01/2022,SJA Lampung,Rajungan (Meat),07/01/2022
3,DO/DOB/20220107/093/1,93,07/01/2022,SJA Lampung,Rajungan (Meat),08/01/2022
4,DO/DOB/20220108/094/1,94,08/01/2022,Nirwana Segara,Rajungan (Meat),09/01/2022
5,DO/DOB/20220109/095/1,95,09/01/2022,Nirwana Segara,Rajungan (Meat),10/01/2022
6,DO/DOB/20220110/096/1,96,10/01/2022,Nirwana Segara,Rajungan (Meat),11/01/2022
7,DO/DOB/20220111/097/1,97,11/01/2022,Nirwana Segara,Rajungan (Meat),12/01/2022
8,DO/DOB/20220113/098/1,98,None,Nirwana Segara,Rajungan (Meat),None
9,DO/DOB/20220114/099/1,99,None,Nirwana Segara,Rajungan (Meat),None


In [55]:
#Ukuran data (jumlah baris, jumlah kolom)
df_DOB_Delivery.shape

(10, 6)

In [56]:
#Summary data info
df_DOB_Delivery.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 6 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   Norm Helper               10 non-null     object
 1   Delivery Number           10 non-null     int64 
 2   Tanggal Delivery dari MP  8 non-null      object
 3   Buyer Name                10 non-null     object
 4   Product Kirim             10 non-null     object
 5   Tanggal Diterima Buyer    8 non-null      object
dtypes: int64(1), object(5)
memory usage: 608.0+ bytes


Kesimpulan :

---
- Terdapat missing value (data hilang/ data kosong) pada kolom `Tanggal Delivery dari MP` dan `Tanggal Diterima Buyer`
- Kolom `Tanggal Delivery dari MP` dan `Tanggal Diterima Buyer` seharusnya berformat datetime


In [57]:
#Data cleaning : mengsisi missing value dengan nilai tanggal sebenarnya pada data di spreadsheet
df_DOB_Delivery.iloc[8, 2] = '13/01/2022'
df_DOB_Delivery.iloc[9, 2] = '14/01/2022'
df_DOB_Delivery.iloc[8, 5] = '14/01/2022'
df_DOB_Delivery.iloc[9, 5] = '15/01/2022'

In [58]:
#Data transformation : mengubah tipe data menjadi datetime
df_DOB_Delivery['Tanggal Delivery dari MP'] = pd.to_datetime(df_DOB_Delivery['Tanggal Delivery dari MP'], dayfirst=True)
df_DOB_Delivery['Tanggal Diterima Buyer'] = pd.to_datetime(df_DOB_Delivery['Tanggal Diterima Buyer'], dayfirst=True)

In [59]:
#Preview Data After
df_DOB_Delivery.head(10)

,Norm Helper,Delivery Number,Tanggal Delivery dari MP,Buyer Name,Product Kirim,Tanggal Diterima Buyer
0,DO/DOB/20220104/090/1,90,2022-01-04,SJA Lampung,Rajungan (Meat),2022-01-05
1,DO/DOB/20220105/091/1,91,2022-01-05,SJA Lampung,Rajungan (Meat),2022-01-06
2,DO/DOB/20220106/092/1,92,2022-01-06,SJA Lampung,Rajungan (Meat),2022-01-07
3,DO/DOB/20220107/093/1,93,2022-01-07,SJA Lampung,Rajungan (Meat),2022-01-08
4,DO/DOB/20220108/094/1,94,2022-01-08,Nirwana Segara,Rajungan (Meat),2022-01-09
5,DO/DOB/20220109/095/1,95,2022-01-09,Nirwana Segara,Rajungan (Meat),2022-01-10
6,DO/DOB/20220110/096/1,96,2022-01-10,Nirwana Segara,Rajungan (Meat),2022-01-11
7,DO/DOB/20220111/097/1,97,2022-01-11,Nirwana Segara,Rajungan (Meat),2022-01-12
8,DO/DOB/20220113/098/1,98,2022-01-13,Nirwana Segara,Rajungan (Meat),2022-01-14
9,DO/DOB/20220114/099/1,99,2022-01-14,Nirwana Segara,Rajungan (Meat),2022-01-15


In [60]:
#Check tipe data
df_DOB_Delivery.dtypes

Norm Helper                         object
Delivery Number                      int64
Tanggal Delivery dari MP    datetime64[ns]
Buyer Name                          object
Product Kirim                       object
Tanggal Diterima Buyer      datetime64[ns]
dtype: object

In [61]:
#Check duplicated data
df_DOB_Delivery.duplicated(subset='Norm Helper', keep='first').sum()

0

### **DOB Buying**

In [62]:
#Read data csv
df5 = pd.read_csv(DOB_Buying)

In [63]:
#Melihat nama kolom
df5.columns

Index(['PO Number', 'Date Helper', 'Norm Helper', 'Date', 'Month', 'Year',
       'Nama Nelayan/Supplier', 'Type', 'Source Place', 'Category', 'Items',
       'Size', 'Code', 'Quantity', 'Kupon', 'Price', 'Amount', 'ID Nelayan',
       'Sequence 1', 'Sequence 2'],
      dtype='object')

In [64]:
#Data transformation : mengubah extention file yang semula CSV menjadi Parquet
df5.to_parquet('DOB_Buying.parquet')
#Menyimpan file parquet di Google Drive
!cp DOB_Buying.parquet "drive/My Drive/"

In [65]:
#Read data parquet
df_DOB_Buying = pd.read_parquet('DOB_Buying.parquet')

In [66]:
#Preview data
df_DOB_Buying.head()

,PO Number,Date Helper,Norm Helper,Date,Month,Year,Nama Nelayan/Supplier,Type,Source Place,Category,Items,Size,Code,Quantity,Kupon,Price,Amount,ID Nelayan,Sequence 1,Sequence 2
0,PO/DOB/20220103/001,03/01/2022,DO/DOB/20220104/090/1,03/01/2022,Jan,2022,MP Kaiwabar,Satelite,Kaiwabar,Rajungan,Rajungan,NaN,RC,70,NaN,35.0,2.450.000,NaN,1,1
1,PO/DOB/20220103/002,03/01/2022,DO/DOB/20220104/090/1,03/01/2022,Jan,2022,Inco,Supplier,Kepulauan Aru,Rajungan,Rajungan,NaN,MEAT,"14,4",NaN,355.0,5.112.000,NaN,2,2
2,PO/DOB/20220104/001,04/01/2022,DO/DOB/20220105/091/1,04/01/2022,Jan,2022,MP Kaiwabar,Satelite,Kaiwabar,Rajungan,Rajungan,NaN,RC,100,NaN,35.0,3.500.000,NaN,1,1
3,PO/DOB/20220104/002,04/01/2022,DO/DOB/20220105/091/1,04/01/2022,Jan,2022,Inco,Supplier,Kepulauan Aru,Rajungan,Rajungan,NaN,MEAT,"9,86",NaN,355.0,3.500.300,NaN,2,2
4,PO/DOB/20220105/001,05/01/2022,DO/DOB/20220106/092/1,05/01/2022,Jan,2022,MP Kaiwabar,Satelite,Kaiwabar,Rajungan,Rajungan,NaN,RC,20,NaN,35.0,700.000,NaN,1,1


In [67]:
#Ukuran data (jumlah baris, jumlah kolom)
df_DOB_Buying.shape

(5, 20)

In [68]:
#Summary data info
df_DOB_Buying.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 20 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   PO Number              5 non-null      object 
 1   Date Helper            5 non-null      object 
 2   Norm Helper            5 non-null      object 
 3   Date                   5 non-null      object 
 4   Month                  5 non-null      object 
 5   Year                   5 non-null      int64  
 6   Nama Nelayan/Supplier  5 non-null      object 
 7   Type                   5 non-null      object 
 8   Source Place           5 non-null      object 
 9   Category               5 non-null      object 
 10  Items                  5 non-null      object 
 11  Size                   0 non-null      float64
 12  Code                   5 non-null      object 
 13  Quantity               5 non-null      object 
 14  Kupon                  0 non-null      float64
 15  Price     

Kesimpulan :

---
- Terdapat missing value (data hilang/ data kosong) pada kolom `Size`, `Kupon`, dan `ID Nelayan`
- Kolom `ID Nelayan` menunjukkan sebuah ID yang dalam database seharusnya memiliki tipe interger
- Kolom `Date Helper` dan `Date` seharusnya berformat datetime
- Kolom `Price` memiliki data yang seharusnya menunjukkan harga dalam ribuan (contoh: 35000)
- Kolom `Amount` yang menunjukkan total harga (quantity*price) memiliki tipe data yang salah yaitu object, seharusnya berformat number (int)
- Kolom `Quantity` seharusnya memiliki tipe number (float/int)

In [69]:
#Data cleaning : mengisi missing value dengan nilai 0 (karena memang tidak diketahui)
df_DOB_Buying = df_DOB_Buying.fillna(0)

In [70]:
#Data transformation : mengubah tipe data dari float --> int
df_DOB_Buying['ID Nelayan'] = df_DOB_Buying['ID Nelayan'].astype('int64')

In [71]:
#Data transformation : mengubah tipe data menjadi datetime
df_DOB_Buying['Date Helper'] = pd.to_datetime(df_DOB_Buying['Date Helper'], dayfirst=True)
df_DOB_Buying['Date'] = pd.to_datetime(df_DOB_Buying['Date'], dayfirst=True)

In [72]:
#Data transformation : mengubah tipe data dari float --> string
df_DOB_Buying['Price'] = df_DOB_Buying['Price'].astype('string')

In [73]:
#Data transformation : mengubah nilai price dalam bentuk ribuan
N = 2
numbers = df_DOB_Buying['Price'].tolist()
df_DOB_Buying['Price'] = [str(num).ljust(N + len(num), '0') for num in numbers]

In [74]:
#Data transformation : mengubah tipe data dari string --> int
df_DOB_Buying['Price'] = df_DOB_Buying['Price'].str.replace('.','')
df_DOB_Buying['Price'] = df_DOB_Buying['Price'].astype('int64')

<ipython-input-74-46eb7856ec79>:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df_DOB_Buying['Price'] = df_DOB_Buying['Price'].str.replace('.','')


In [75]:
#Data transformation : mengubah tipe data dari object --> int
df_DOB_Buying['Amount'] = df_DOB_Buying['Amount'].str.replace('.','')
df_DOB_Buying['Amount'] = df_DOB_Buying['Amount'].astype('int64')

<ipython-input-75-6d307b2f619c>:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df_DOB_Buying['Amount'] = df_DOB_Buying['Amount'].str.replace('.','')


In [76]:
#Data transformation : mengubah tipe data dari object --> float
df_DOB_Buying['Quantity'] = df_DOB_Buying['Quantity'].str.replace(',','.')
df_DOB_Buying['Quantity'] = df_DOB_Buying['Quantity'].astype('float64')

In [77]:
#Preview Data After
df_DOB_Buying.head()

,PO Number,Date Helper,Norm Helper,Date,Month,Year,Nama Nelayan/Supplier,Type,Source Place,Category,Items,Size,Code,Quantity,Kupon,Price,Amount,ID Nelayan,Sequence 1,Sequence 2
0,PO/DOB/20220103/001,2022-01-03,DO/DOB/20220104/090/1,2022-01-03,Jan,2022,MP Kaiwabar,Satelite,Kaiwabar,Rajungan,Rajungan,0.0,RC,70.00,0.0,35000,2450000,0,1,1
1,PO/DOB/20220103/002,2022-01-03,DO/DOB/20220104/090/1,2022-01-03,Jan,2022,Inco,Supplier,Kepulauan Aru,Rajungan,Rajungan,0.0,MEAT,14.40,0.0,355000,5112000,0,2,2
2,PO/DOB/20220104/001,2022-01-04,DO/DOB/20220105/091/1,2022-01-04,Jan,2022,MP Kaiwabar,Satelite,Kaiwabar,Rajungan,Rajungan,0.0,RC,100.00,0.0,35000,3500000,0,1,1
3,PO/DOB/20220104/002,2022-01-04,DO/DOB/20220105/091/1,2022-01-04,Jan,2022,Inco,Supplier,Kepulauan Aru,Rajungan,Rajungan,0.0,MEAT,9.86,0.0,355000,3500300,0,2,2
4,PO/DOB/20220105/001,2022-01-05,DO/DOB/20220106/092/1,2022-01-05,Jan,2022,MP Kaiwabar,Satelite,Kaiwabar,Rajungan,Rajungan,0.0,RC,20.00,0.0,35000,700000,0,1,1


In [78]:
#Check tipe data
df_DOB_Buying.dtypes

PO Number                        object
Date Helper              datetime64[ns]
Norm Helper                      object
Date                     datetime64[ns]
Month                            object
Year                              int64
Nama Nelayan/Supplier            object
Type                             object
Source Place                     object
Category                         object
Items                            object
Size                            float64
Code                             object
Quantity                        float64
Kupon                           float64
Price                             int64
Amount                            int64
ID Nelayan                        int64
Sequence 1                        int64
Sequence 2                        int64
dtype: object

In [79]:
#Check duplicated data
df_DOB_Buying.duplicated(subset='PO Number', keep='first').sum()

0

### **DOB Processing**

In [80]:
#Read data csv
df6 = pd.read_csv(DOB_Processing)

In [81]:
#Melihat nama kolom
df6.columns

Index(['PR No', 'Date helper', 'Norm Helper', 'Date', 'RM', 'RC', '%',
       'Total RC', 'RC Dback', '%.1', 'Total RC Dback', 'Meat', '%.2',
       'Total Meat', 'Picker Price', 'Amount', 'Sequence 1', 'Sequence 2',
       'Unnamed: 18', 'Unnamed: 19', 'Unnamed: 20', 'Unnamed: 21',
       'Unnamed: 22', 'Unnamed: 23', 'Unnamed: 24', 'Unnamed: 25'],
      dtype='object')

In [82]:
cols_to_drop = [ 'Unnamed: 18', 'Unnamed: 19', 'Unnamed: 20', 'Unnamed: 21', 'Unnamed: 22', 'Unnamed: 23', 'Unnamed: 24', 'Unnamed: 25' ]

In [83]:
#Data cleaning : menghapus semua kolom Unnamed
df6 = df6.drop(cols_to_drop, axis=1)
df6.columns

Index(['PR No', 'Date helper', 'Norm Helper', 'Date', 'RM', 'RC', '%',
       'Total RC', 'RC Dback', '%.1', 'Total RC Dback', 'Meat', '%.2',
       'Total Meat', 'Picker Price', 'Amount', 'Sequence 1', 'Sequence 2'],
      dtype='object')

In [84]:
#Data transformation : mengubah extention file yang semula CSV menjadi Parquet
df6.to_parquet('DOB_Processing.parquet')
#Menyimpan file parquet di Google Drive
!cp DOB_Processing.parquet "drive/My Drive/"

In [85]:
#Read data parquet
df_DOB_Processing = pd.read_parquet('DOB_Processing.parquet')

In [86]:
#Preview data
df_DOB_Processing.head()

,PR No,Date helper,Norm Helper,Date,RM,RC,%,Total RC,RC Dback,%.1,Total RC Dback,Meat,%.2,Total Meat,Picker Price,Amount,Sequence 1,Sequence 2
0,PR/DOB/20220103/001,03/01/2022,DO/DOB/20220104/090/1,03/01/2022,NaN,70,100%,70,NaN,NaN,NaN,"19,35",28%,"19,35",25.0,870.750,1,1
1,PR/DOB/20220104/001,04/01/2022,DO/DOB/20220105/091/1,04/01/2022,NaN,100,100%,100,NaN,NaN,NaN,"26,21",26%,"26,21",25.0,1.179.450,1,1
2,PR/DOB/20220105/001,05/01/2022,DO/DOB/20220106/092/1,05/01/2022,NaN,25,100%,25,NaN,NaN,NaN,"7,40",30%,"7,40",25.0,333.000,1,1
3,PR/DOB/20220107/001,07/01/2022,DO/DOB/20220108/094/1,07/01/2022,NaN,"142,6",100%,"142,6",NaN,NaN,NaN,"35,65",25%,"35,65",25.0,1.604.250,1,1
4,PR/DOB/20220108/001,08/01/2022,DO/DOB/20220109/095/1,08/01/2022,NaN,"158,21",100%,"158,21",NaN,NaN,NaN,"38,05",24%,"38,05",25.0,1.712.250,1,1


In [87]:
#Ukuran data (jumlah baris, jumlah kolom)
df_DOB_Processing.shape

(5, 18)

In [88]:
#Summary data info
df_DOB_Processing.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 18 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   PR No           5 non-null      object 
 1   Date helper     5 non-null      object 
 2   Norm Helper     5 non-null      object 
 3   Date            5 non-null      object 
 4   RM              0 non-null      float64
 5   RC              5 non-null      object 
 6   %               5 non-null      object 
 7   Total RC        5 non-null      object 
 8   RC Dback        0 non-null      float64
 9   %.1             0 non-null      float64
 10  Total RC Dback  0 non-null      float64
 11  Meat            5 non-null      object 
 12  %.2             5 non-null      object 
 13  Total Meat      5 non-null      object 
 14  Picker Price    5 non-null      float64
 15  Amount          5 non-null      object 
 16  Sequence 1      5 non-null      int64  
 17  Sequence 2      5 non-null      int64  

Kesimpulan :

---
- Terdapat missing value (data hilang/ data kosong) pada kolom `RM`, `RC Dback`, `%.1`, dan `Total RC Dback`
- Kolom `Date Helper` dan `Date` seharusnya berformat datetime
- Kolom `RC`, `%`, `Total RC`, `Meat`, `%.2`, dan `Total Meat` seharusnya bertipe number (float)
- Kolom `Amount` yang menunjukkan total harga  memiliki tipe data yang salah yaitu object, seharusnya berformat number (int)

In [89]:
#Data cleaning : mengisi missing value dengan nilai 0 (karena memang tidak diketahui)
df_DOB_Processing = df_DOB_Processing.fillna(0)

In [90]:
#Data transformation : mengubah tipe data menjadi datetime
df_DOB_Processing['Date helper'] = pd.to_datetime(df_DOB_Processing['Date helper'], dayfirst=True)
df_DOB_Processing['Date'] = pd.to_datetime(df_DOB_Processing['Date'], dayfirst=True)

In [91]:
#Data transformation : mengubah tipe data dari object --> float
df_DOB_Processing['RC'] = df_DOB_Processing['RC'].str.replace(',','.')
df_DOB_Processing['RC'] = df_DOB_Processing['RC'].astype('float64')
df_DOB_Processing['Total RC'] = df_DOB_Processing['Total RC'].str.replace(',','.')
df_DOB_Processing['Total RC'] = df_DOB_Processing['Total RC'].astype('float64')
df_DOB_Processing['Total Meat'] = df_DOB_Processing['Total Meat'].str.replace(',','.')
df_DOB_Processing['Total Meat'] = df_DOB_Processing['Total Meat'].astype('float64')
df_DOB_Processing['Meat'] = df_DOB_Processing['Meat'].str.replace(',','.')
df_DOB_Processing['Meat'] = df_DOB_Processing['Meat'].astype('float64')

In [92]:
#Data transformation : mengubah tipe data dari object --> int
df_DOB_Processing['Amount'] = df_DOB_Processing['Amount'].str.replace('.','')
df_DOB_Processing['Amount'] = df_DOB_Processing['Amount'].astype('int64')

<ipython-input-92-717b22232ec1>:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df_DOB_Processing['Amount'] = df_DOB_Processing['Amount'].str.replace('.','')


In [93]:
#Data transformation : mengubah percentage menjadi float
df_DOB_Processing['%'] = (df_DOB_Processing['%'].str.replace('%',''))
df_DOB_Processing['%'] = df_DOB_Processing['%'].astype('float64')
df_DOB_Processing['%'] = df_DOB_Processing['%']/100
df_DOB_Processing['%.2'] = df_DOB_Processing['%.2'].str.replace('%','')
df_DOB_Processing['%.2'] = df_DOB_Processing['%.2'].astype('float64')
df_DOB_Processing['%.2'] = df_DOB_Processing['%.2']/100

In [94]:
#Preview Data After
df_DOB_Processing.head()

,PR No,Date helper,Norm Helper,Date,RM,RC,%,Total RC,RC Dback,%.1,Total RC Dback,Meat,%.2,Total Meat,Picker Price,Amount,Sequence 1,Sequence 2
0,PR/DOB/20220103/001,2022-01-03,DO/DOB/20220104/090/1,2022-01-03,0.0,70.00,1.0,70.00,0.0,0.0,0.0,19.35,0.28,19.35,25.0,870750,1,1
1,PR/DOB/20220104/001,2022-01-04,DO/DOB/20220105/091/1,2022-01-04,0.0,100.00,1.0,100.00,0.0,0.0,0.0,26.21,0.26,26.21,25.0,1179450,1,1
2,PR/DOB/20220105/001,2022-01-05,DO/DOB/20220106/092/1,2022-01-05,0.0,25.00,1.0,25.00,0.0,0.0,0.0,7.40,0.30,7.40,25.0,333000,1,1
3,PR/DOB/20220107/001,2022-01-07,DO/DOB/20220108/094/1,2022-01-07,0.0,142.60,1.0,142.60,0.0,0.0,0.0,35.65,0.25,35.65,25.0,1604250,1,1
4,PR/DOB/20220108/001,2022-01-08,DO/DOB/20220109/095/1,2022-01-08,0.0,158.21,1.0,158.21,0.0,0.0,0.0,38.05,0.24,38.05,25.0,1712250,1,1


In [95]:
#Check tipe data
df_DOB_Processing.dtypes

PR No                     object
Date helper       datetime64[ns]
Norm Helper               object
Date              datetime64[ns]
RM                       float64
RC                       float64
%                        float64
Total RC                 float64
RC Dback                 float64
%.1                      float64
Total RC Dback           float64
Meat                     float64
%.2                      float64
Total Meat               float64
Picker Price             float64
Amount                     int64
Sequence 1                 int64
Sequence 2                 int64
dtype: object

In [96]:
#Check duplicated data
df_DOB_Processing.duplicated(subset='PR No', keep='first').sum()

0

## **Join/ Concat Dataframe**

The concat() function can be used to concatenate two Dataframes by adding the rows of one to the other.

In [97]:
#Menggabungkan dataframe dengan transaksi delivery
df_all_delivery = pd.concat([df_BKJ_Delivery, df_DOB_Delivery])
df_all_delivery

,Norm Helper,Delivery Number,Tanggal Delivery dari MP,Buyer Name,Product Kirim,Tanggal Diterima Buyer
0,DO/BKJ/20220305/001/1,1,2022-03-05,PT Prima,Tuna (Whole),2022-03-06
1,DO/BKJ/20220307/002/1,2,2022-03-07,PT Prima,Tuna (Whole),2022-03-08
2,DO/BKJ/20220309/003/1,3,2022-03-09,PT Prima,Tuna (Whole),2022-03-10
0,DO/DOB/20220104/090/1,90,2022-01-04,SJA Lampung,Rajungan (Meat),2022-01-05
1,DO/DOB/20220105/091/1,91,2022-01-05,SJA Lampung,Rajungan (Meat),2022-01-06
2,DO/DOB/20220106/092/1,92,2022-01-06,SJA Lampung,Rajungan (Meat),2022-01-07
3,DO/DOB/20220107/093/1,93,2022-01-07,SJA Lampung,Rajungan (Meat),2022-01-08
4,DO/DOB/20220108/094/1,94,2022-01-08,Nirwana Segara,Rajungan (Meat),2022-01-09
5,DO/DOB/20220109/095/1,95,2022-01-09,Nirwana Segara,Rajungan (Meat),2022-01-10
6,DO/DOB/20220110/096/1,96,2022-01-10,Nirwana Segara,Rajungan (Meat),2022-01-11


In [98]:
#Menggabungkan dataframe dengan transaksi buying
df_all_buying = pd.concat([df_BKJ_Buying, df_DOB_Buying])
df_all_buying

,PO Number,Date Helper,Norm Helper,Date,Month,Year,Nama Nelayan/Supplier,Type,Source Place,Category,Items,Size,Code,Quantity,Kupon,Price,Amount,ID Nelayan,Sequence 1,Sequence 2
0,PO/BKJ/20220305/001,2022-03-05,DO/BKJ/20220305/001/1,2022-03-05,Mar,2022,Nazar,Mini Plant,Bangka Jaya,Ikan Pelagis,Tuna,0.0,TUNA 20,248.00,0.0,29000,10664000,0,1,1
1,PO/BKJ/20220305/002,2022-03-05,DO/BKJ/20220305/001/1,2022-03-05,Mar,2022,Moza,Supplier,Krueng Mane,Ikan Pelagis,Tuna,0.0,TUNA 20,252.00,0.0,29000,10836000,0,2,2
2,PO/BKJ/20220306/001,2022-03-06,DO/BKJ/20220307/002/1,2022-03-06,Mar,2022,Gam,Supplier,Bangka Jaya,Ikan Pelagis,Tuna,0.0,TUNA 20,30.00,0.0,29000,1290000,0,1,1
0,PO/DOB/20220103/001,2022-01-03,DO/DOB/20220104/090/1,2022-01-03,Jan,2022,MP Kaiwabar,Satelite,Kaiwabar,Rajungan,Rajungan,0.0,RC,70.00,0.0,35000,2450000,0,1,1
1,PO/DOB/20220103/002,2022-01-03,DO/DOB/20220104/090/1,2022-01-03,Jan,2022,Inco,Supplier,Kepulauan Aru,Rajungan,Rajungan,0.0,MEAT,14.40,0.0,355000,5112000,0,2,2
2,PO/DOB/20220104/001,2022-01-04,DO/DOB/20220105/091/1,2022-01-04,Jan,2022,MP Kaiwabar,Satelite,Kaiwabar,Rajungan,Rajungan,0.0,RC,100.00,0.0,35000,3500000,0,1,1
3,PO/DOB/20220104/002,2022-01-04,DO/DOB/20220105/091/1,2022-01-04,Jan,2022,Inco,Supplier,Kepulauan Aru,Rajungan,Rajungan,0.0,MEAT,9.86,0.0,355000,3500300,0,2,2
4,PO/DOB/20220105/001,2022-01-05,DO/DOB/20220106/092/1,2022-01-05,Jan,2022,MP Kaiwabar,Satelite,Kaiwabar,Rajungan,Rajungan,0.0,RC,20.00,0.0,35000,700000,0,1,1


In [99]:
#Menggabungkan dataframe dengan transaksi processing
df_all_processing = pd.concat([df_BKJ_Processing, df_DOB_Processing])
df_all_processing

,PR No,Date helper,Norm Helper,Date,RM,RC,%,Total RC,RC Dback,%.1,Total RC Dback,Meat,%.2,Total Meat,Picker Price,Amount,Sequence 1,Sequence 2
0,PR/DOB/20220103/001,2022-01-03,DO/DOB/20220104/090/1,2022-01-03,0.0,70.00,1.0,70.00,0.0,0.0,0.0,19.35,0.28,19.35,25.0,870750,1,1
1,PR/DOB/20220104/001,2022-01-04,DO/DOB/20220105/091/1,2022-01-04,0.0,100.00,1.0,100.00,0.0,0.0,0.0,26.21,0.26,26.21,25.0,1179450,1,1
2,PR/DOB/20220105/001,2022-01-05,DO/DOB/20220106/092/1,2022-01-05,0.0,25.00,1.0,25.00,0.0,0.0,0.0,7.40,0.30,7.40,25.0,333000,1,1
3,PR/DOB/20220107/001,2022-01-07,DO/DOB/20220108/094/1,2022-01-07,0.0,142.60,1.0,142.60,0.0,0.0,0.0,35.65,0.25,35.65,25.0,1604250,1,1
4,PR/DOB/20220108/001,2022-01-08,DO/DOB/20220109/095/1,2022-01-08,0.0,158.21,1.0,158.21,0.0,0.0,0.0,38.05,0.24,38.05,25.0,1712250,1,1


The merge() function is equivalent to the SQL JOIN clause. ‘left’, ‘right’ and ‘inner’ joins are all possible.



In [100]:
#Menggabungkan dataframe dengan transaksi BKJ
df_all_BKJ = pd.merge(pd.merge(df_BKJ_Delivery, df_BKJ_Buying, on='Norm Helper', how='left'), df_BKJ_Processing, on='Norm Helper', how='left')
df_all_BKJ

,Norm Helper,Delivery Number,Tanggal Delivery dari MP,Buyer Name,Product Kirim,Tanggal Diterima Buyer,PO Number,Date Helper,Date_x,Month,Year,Nama Nelayan/Supplier,Type,Source Place,Category,Items,Size,Code,Quantity,Kupon,Price,Amount_x,ID Nelayan,Sequence 1_x,Sequence 2_x,PR No,Date helper,Date_y,RM,RC,%,Total RC,RC Dback,%.1,Total RC Dback,Meat,%.2,Total Meat,Picker Price,Amount_y,Sequence 1_y,Sequence 2_y
0,DO/BKJ/20220305/001/1,1,2022-03-05,PT Prima,Tuna (Whole),2022-03-06,PO/BKJ/20220305/001,2022-03-05,2022-03-05,Mar,2022.0,Nazar,Mini Plant,Bangka Jaya,Ikan Pelagis,Tuna,0.0,TUNA 20,248.0,0.0,29000.0,10664000.0,0.0,1.0,1.0,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,DO/BKJ/20220305/001/1,1,2022-03-05,PT Prima,Tuna (Whole),2022-03-06,PO/BKJ/20220305/002,2022-03-05,2022-03-05,Mar,2022.0,Moza,Supplier,Krueng Mane,Ikan Pelagis,Tuna,0.0,TUNA 20,252.0,0.0,29000.0,10836000.0,0.0,2.0,2.0,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,DO/BKJ/20220307/002/1,2,2022-03-07,PT Prima,Tuna (Whole),2022-03-08,PO/BKJ/20220306/001,2022-03-06,2022-03-06,Mar,2022.0,Gam,Supplier,Bangka Jaya,Ikan Pelagis,Tuna,0.0,TUNA 20,30.0,0.0,29000.0,1290000.0,0.0,1.0,1.0,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,DO/BKJ/20220309/003/1,3,2022-03-09,PT Prima,Tuna (Whole),2022-03-10,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [101]:
df_all_BKJ.shape

(4, 42)

In [102]:
#Menggabungkan dataframe dengan transaksi DOB
df_all_DOB = pd.merge(pd.merge(df_DOB_Delivery, df_DOB_Buying, on='Norm Helper', how='left'), df_DOB_Processing, on='Norm Helper', how='left')
df_all_DOB

,Norm Helper,Delivery Number,Tanggal Delivery dari MP,Buyer Name,Product Kirim,Tanggal Diterima Buyer,PO Number,Date Helper,Date_x,Month,Year,Nama Nelayan/Supplier,Type,Source Place,Category,Items,Size,Code,Quantity,Kupon,Price,Amount_x,ID Nelayan,Sequence 1_x,Sequence 2_x,PR No,Date helper,Date_y,RM,RC,%,Total RC,RC Dback,%.1,Total RC Dback,Meat,%.2,Total Meat,Picker Price,Amount_y,Sequence 1_y,Sequence 2_y
0,DO/DOB/20220104/090/1,90,2022-01-04,SJA Lampung,Rajungan (Meat),2022-01-05,PO/DOB/20220103/001,2022-01-03,2022-01-03,Jan,2022.0,MP Kaiwabar,Satelite,Kaiwabar,Rajungan,Rajungan,0.0,RC,70.00,0.0,35000.0,2450000.0,0.0,1.0,1.0,PR/DOB/20220103/001,2022-01-03,2022-01-03,0.0,70.00,1.0,70.00,0.0,0.0,0.0,19.35,0.28,19.35,25.0,870750.0,1.0,1.0
1,DO/DOB/20220104/090/1,90,2022-01-04,SJA Lampung,Rajungan (Meat),2022-01-05,PO/DOB/20220103/002,2022-01-03,2022-01-03,Jan,2022.0,Inco,Supplier,Kepulauan Aru,Rajungan,Rajungan,0.0,MEAT,14.40,0.0,355000.0,5112000.0,0.0,2.0,2.0,PR/DOB/20220103/001,2022-01-03,2022-01-03,0.0,70.00,1.0,70.00,0.0,0.0,0.0,19.35,0.28,19.35,25.0,870750.0,1.0,1.0
2,DO/DOB/20220105/091/1,91,2022-01-05,SJA Lampung,Rajungan (Meat),2022-01-06,PO/DOB/20220104/001,2022-01-04,2022-01-04,Jan,2022.0,MP Kaiwabar,Satelite,Kaiwabar,Rajungan,Rajungan,0.0,RC,100.00,0.0,35000.0,3500000.0,0.0,1.0,1.0,PR/DOB/20220104/001,2022-01-04,2022-01-04,0.0,100.00,1.0,100.00,0.0,0.0,0.0,26.21,0.26,26.21,25.0,1179450.0,1.0,1.0
3,DO/DOB/20220105/091/1,91,2022-01-05,SJA Lampung,Rajungan (Meat),2022-01-06,PO/DOB/20220104/002,2022-01-04,2022-01-04,Jan,2022.0,Inco,Supplier,Kepulauan Aru,Rajungan,Rajungan,0.0,MEAT,9.86,0.0,355000.0,3500300.0,0.0,2.0,2.0,PR/DOB/20220104/001,2022-01-04,2022-01-04,0.0,100.00,1.0,100.00,0.0,0.0,0.0,26.21,0.26,26.21,25.0,1179450.0,1.0,1.0
4,DO/DOB/20220106/092/1,92,2022-01-06,SJA Lampung,Rajungan (Meat),2022-01-07,PO/DOB/20220105/001,2022-01-05,2022-01-05,Jan,2022.0,MP Kaiwabar,Satelite,Kaiwabar,Rajungan,Rajungan,0.0,RC,20.00,0.0,35000.0,700000.0,0.0,1.0,1.0,PR/DOB/20220105/001,2022-01-05,2022-01-05,0.0,25.00,1.0,25.00,0.0,0.0,0.0,7.40,0.30,7.40,25.0,333000.0,1.0,1.0
5,DO/DOB/20220107/093/1,93,2022-01-07,SJA Lampung,Rajungan (Meat),2022-01-08,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,DO/DOB/20220108/094/1,94,2022-01-08,Nirwana Segara,Rajungan (Meat),2022-01-09,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PR/DOB/20220107/001,2022-01-07,2022-01-07,0.0,142.60,1.0,142.60,0.0,0.0,0.0,35.65,0.25,35.65,25.0,1604250.0,1.0,1.0
7,DO/DOB/20220109/095/1,95,2022-01-09,Nirwana Segara,Rajungan (Meat),2022-01-10,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PR/DOB/20220108/001,2022-01-08,2022-01-08,0.0,158.21,1.0,158.21,0.0,0.0,0.0,38.05,0.24,38.05,25.0,1712250.0,1.0,1.0
8,DO/DOB/20220110/096/1,96,2022-01-10,Nirwana Segara,Rajungan (Meat),2022-01-11,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,DO/DOB/20220111/097/1,97,2022-01-11,Nirwana Segara,Rajungan (Meat),2022-01-12,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [103]:
df_all_DOB.shape

(12, 42)

In [104]:
#Menggabungkan semua dataframe
df_all = pd.concat([df_all_BKJ, df_all_DOB])
df_all

,Norm Helper,Delivery Number,Tanggal Delivery dari MP,Buyer Name,Product Kirim,Tanggal Diterima Buyer,PO Number,Date Helper,Date_x,Month,Year,Nama Nelayan/Supplier,Type,Source Place,Category,Items,Size,Code,Quantity,Kupon,Price,Amount_x,ID Nelayan,Sequence 1_x,Sequence 2_x,PR No,Date helper,Date_y,RM,RC,%,Total RC,RC Dback,%.1,Total RC Dback,Meat,%.2,Total Meat,Picker Price,Amount_y,Sequence 1_y,Sequence 2_y
0,DO/BKJ/20220305/001/1,1,2022-03-05,PT Prima,Tuna (Whole),2022-03-06,PO/BKJ/20220305/001,2022-03-05,2022-03-05,Mar,2022.0,Nazar,Mini Plant,Bangka Jaya,Ikan Pelagis,Tuna,0.0,TUNA 20,248.00,0.0,29000.0,10664000.0,0.0,1.0,1.0,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,DO/BKJ/20220305/001/1,1,2022-03-05,PT Prima,Tuna (Whole),2022-03-06,PO/BKJ/20220305/002,2022-03-05,2022-03-05,Mar,2022.0,Moza,Supplier,Krueng Mane,Ikan Pelagis,Tuna,0.0,TUNA 20,252.00,0.0,29000.0,10836000.0,0.0,2.0,2.0,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,DO/BKJ/20220307/002/1,2,2022-03-07,PT Prima,Tuna (Whole),2022-03-08,PO/BKJ/20220306/001,2022-03-06,2022-03-06,Mar,2022.0,Gam,Supplier,Bangka Jaya,Ikan Pelagis,Tuna,0.0,TUNA 20,30.00,0.0,29000.0,1290000.0,0.0,1.0,1.0,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,DO/BKJ/20220309/003/1,3,2022-03-09,PT Prima,Tuna (Whole),2022-03-10,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,DO/DOB/20220104/090/1,90,2022-01-04,SJA Lampung,Rajungan (Meat),2022-01-05,PO/DOB/20220103/001,2022-01-03,2022-01-03,Jan,2022.0,MP Kaiwabar,Satelite,Kaiwabar,Rajungan,Rajungan,0.0,RC,70.00,0.0,35000.0,2450000.0,0.0,1.0,1.0,PR/DOB/20220103/001,2022-01-03,2022-01-03,0.0,70.00,1.0,70.00,0.0,0.0,0.0,19.35,0.28,19.35,25.0,870750.0,1.0,1.0
1,DO/DOB/20220104/090/1,90,2022-01-04,SJA Lampung,Rajungan (Meat),2022-01-05,PO/DOB/20220103/002,2022-01-03,2022-01-03,Jan,2022.0,Inco,Supplier,Kepulauan Aru,Rajungan,Rajungan,0.0,MEAT,14.40,0.0,355000.0,5112000.0,0.0,2.0,2.0,PR/DOB/20220103/001,2022-01-03,2022-01-03,0.0,70.00,1.0,70.00,0.0,0.0,0.0,19.35,0.28,19.35,25.0,870750.0,1.0,1.0
2,DO/DOB/20220105/091/1,91,2022-01-05,SJA Lampung,Rajungan (Meat),2022-01-06,PO/DOB/20220104/001,2022-01-04,2022-01-04,Jan,2022.0,MP Kaiwabar,Satelite,Kaiwabar,Rajungan,Rajungan,0.0,RC,100.00,0.0,35000.0,3500000.0,0.0,1.0,1.0,PR/DOB/20220104/001,2022-01-04,2022-01-04,0.0,100.00,1.0,100.00,0.0,0.0,0.0,26.21,0.26,26.21,25.0,1179450.0,1.0,1.0
3,DO/DOB/20220105/091/1,91,2022-01-05,SJA Lampung,Rajungan (Meat),2022-01-06,PO/DOB/20220104/002,2022-01-04,2022-01-04,Jan,2022.0,Inco,Supplier,Kepulauan Aru,Rajungan,Rajungan,0.0,MEAT,9.86,0.0,355000.0,3500300.0,0.0,2.0,2.0,PR/DOB/20220104/001,2022-01-04,2022-01-04,0.0,100.00,1.0,100.00,0.0,0.0,0.0,26.21,0.26,26.21,25.0,1179450.0,1.0,1.0
4,DO/DOB/20220106/092/1,92,2022-01-06,SJA Lampung,Rajungan (Meat),2022-01-07,PO/DOB/20220105/001,2022-01-05,2022-01-05,Jan,2022.0,MP Kaiwabar,Satelite,Kaiwabar,Rajungan,Rajungan,0.0,RC,20.00,0.0,35000.0,700000.0,0.0,1.0,1.0,PR/DOB/20220105/001,2022-01-05,2022-01-05,0.0,25.00,1.0,25.00,0.0,0.0,0.0,7.40,0.30,7.40,25.0,333000.0,1.0,1.0
5,DO/DOB/20220107/093/1,93,2022-01-07,SJA Lampung,Rajungan (Meat),2022-01-08,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [105]:
df_all.shape

(16, 42)

In [106]:
#Save file to parquet
df_all.to_parquet('transactions_all.parquet', index=False)
df_all_BKJ.to_parquet('transactions_BKJ.parquet', index=False)
df_all_DOB.to_parquet('transactions_DOB.parquet', index=False)
df_all_processing.to_parquet('transactions_processing.parquet', index=False)
df_all_buying.to_parquet('transactions_buying.parquet', index=False)
df_all_delivery.to_parquet('transactions_delivery.parquet', index=False)
#Menyimpan file parquet di Google Drive
!cp transactions_all.parquet "drive/My Drive/Output"
!cp transactions_BKJ.parquet "drive/My Drive/Output"
!cp transactions_DOB.parquet "drive/My Drive/Output"
!cp transactions_processing.parquet "drive/My Drive/Output"
!cp transactions_buying.parquet "drive/My Drive/Output"
!cp transactions_delivery.parquet "drive/My Drive/Output"